## 0. Import Packages:
First, we import all the packages we want to use in our implementation:
* A library to use operating system dependent functionality
* Package imaging library to deal with images in Python (PIL)
* Package to find all paths which matches a specified pattern (glob)
* Numpy Package (numpy)
* PyTorch Framework (torch)
* Neural Network Library of PyTorch (torch.nn)
* PyTorch Optimisation Package (torch.optim)
* PyTorch dataset loader package (torchvision.datasets)
* PyTorch package for image preprocessing (torchvision.transforms)
* A library to loop through the csv file (pandas)
* A library for image manpulation (opencv)

In [95]:
import os
from PIL import Image
from glob import glob
from time import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets as dsets
from torchvision import transforms as trans
from torch.utils.data import Dataset, DataLoader, Subset

import pandas as pd
import cv2

from sklearn.model_selection import train_test_split
import shutil
import random

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

## 3. Set Hyperparameters:
Hyperparameters are settings that can be tuned to control the behaviour of the model.

In [96]:
# Training Hyperparameters (Note: These values are not the optimal ones)
batch_size = 16
learning_rate = 0.1
itr = 20 
transformType = 1

In [97]:
def get_max_image_size(root_folder):
    max_width, max_height = 0, 0
    image_paths = glob(os.path.join(root_folder, '**', '*.png'), recursive=True)

    for path in image_paths:
        with Image.open(path) as img:
            width, height = img.size
            max_width = max(max_width, width)
            max_height = max(max_height, height)

    return max_width, max_height

In [98]:
split_folder = 'dataset/split_dataset/'
image_size  = None
SizeY = 0

# Define transformation
transforms_resize = trans.Compose([trans.Grayscale(), trans.Resize([32,32]), trans.ToTensor(), trans.Normalize(mean=(0.5,), std = (0.5,))])

max_width, max_height = get_max_image_size(split_folder)
print(f"Max Image Size: {max_width}x{max_height}")
target_size = (max_width, max_height)

# Define the CenterCrop transform
transforms_pad = trans.Compose([trans.Grayscale(), trans.CenterCrop(target_size), trans.ToTensor(),trans.Normalize(mean=(0.5,), std = (0.5,))
])

transforms_no_resize = trans.Compose([trans.Grayscale(), trans.ToTensor(),trans.Normalize(mean=(0.5,), std = (0.5,))
])

transforms = None


if transformType == 1:
    transforms = transforms_resize
    image_size = (32, 32)
elif transformType == 2:
    transforms = transforms_pad
    image_size = (max_width, max_height)
else:
    transforms = transforms_no_resize
    image_size = (0, 0)
    


Max Image Size: 304x641


In [99]:
class ConvNet(nn.Module):
    def __init__(self, input_size, TransformType):
        super(ConvNet, self).__init__() 

        self.input_size = input_size
        self.TransformType = TransformType
        
        self.conv1 = nn.Conv2d(in_channels=1,
        	                   out_channels=6,
            				   kernel_size=(5,5),
            				   stride=(1,1))
        self.pool2 = nn.MaxPool2d(kernel_size = (2,2))
        self.conv3 = nn.Conv2d(in_channels=6,
        	                   out_channels=16,
            				   kernel_size=(5,5),
            				   stride=(1,1))
        self.pool4 = nn.MaxPool2d(kernel_size = (2,2))
        self.conv5 = nn.Conv2d(in_channels=16,
        	                   out_channels=120,
            				   kernel_size=(5,5),
            				   stride=(1,1))

        if self.TransformType == 3:
            self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
            self.fc6 = nn.Linear(in_features=120,
            			     out_features=60)
        elif self.TransformType == 4:
            self.global_pool = nn.AdaptiveMaxPool2d((1, 1))
            self.fc6 = nn.Linear(in_features=120,
            			     out_features=60)
        else:
            self.flattened_size = self._get_flattened_size()
            self.fc6 = nn.Linear(in_features=self.flattened_size,
            			     out_features=60)
            
        self.fc7 = nn.Linear(in_features=60,
            			     out_features=20)
        self.fc8 = nn.Linear(in_features=20,
            			     out_features=4)
    def _get_flattened_size(self):
        # Create a dummy input to compute the size after convolutional and pooling layers
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, *self.input_size)  # Batch size of 1, 1 channel
            dummy_output = self.pool4(self.conv3(self.pool2(self.conv1(dummy_input))))
            dummy_output = self.conv5(dummy_output)
            flattened_size = dummy_output.view(1, -1).size(1)  # Flatten and get the size
        return flattened_size
        
    def forward(self,x):
        #x = x.view([-1, 1, SizeX, SizeY])
        x = nn.functional.relu(self.conv1(x))
        x = self.pool2(x)
        x = nn.functional.relu(self.conv3(x))
        x = self.pool4(x)
        x = nn.functional.relu(self.conv5(x))

        if self.TransformType == 3 or self.TransformType == 4:
            x = self.global_pool(x)
            x = x.view(x.size(0), -1)
        else:
            x = x.view(-1, self.flattened_size)
            
        x = nn.functional.relu(self.fc6(x))
        x = nn.functional.relu(self.fc7(x))
        x = self.fc8(x)
        return x

In [100]:
cuda = False
torch.manual_seed(0)
if torch.cuda.is_available() and cuda:
    torch.cuda.manual_seed_all(0)
    FloatType = torch.cuda.FloatTensor
    LongType = torch.cuda.LongTensor
else:
    FloatType = torch.FloatTensor
    LongType = torch.LongTensor

In [101]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight.data)
        m.bias.data.normal_(mean=0,std=1e-2)
    elif isinstance(m, nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight.data)
        m.bias.data.normal_(mean=0,std=1e-2)

In [102]:
class MIDSDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.paths = glob(os.path.join(self.root, '**', "*.png"))

    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        img = self.transform(Image.open(path))
        label = int(path.split(os.path.sep)[-2])
        return img, label

# Create Subsets for PyTorch DataLoader
train_dataset = MIDSDataset(root = os.path.join(split_folder, 'train'), transform= transforms)
val_dataset = MIDSDataset(root = os.path.join(split_folder, 'val'), transform= transforms)
test_dataset = MIDSDataset(root = os.path.join(split_folder, 'test'), transform= transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers = 0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = 0)


# Print dataset sizes
print(f"Train Samples: {len(train_dataset)}, Validation Samples: {len(val_dataset)}, Test Samples: {len(test_dataset)}")


Train Samples: 601, Validation Samples: 75, Test Samples: 78


In [103]:
def train_model(model, optimizer, train_loader, loss_func, epoch, scheduler=None, vis_step = 5):
    # Number of samples with correct classification
    num_hit = 0
    # total size of train data
    total = len(train_loader.dataset)
    # number of batch
    num_batch = np.ceil(total/batch_size)
    accumulative_loss = 0
    # Training loop over batches of data on train dataset
    for batch_idx, (image, labels) in enumerate(train_loader):
        # 1. Clearing previous gradient values.
        optimizer.zero_grad()
        # 2. feeding images to model (forward method will be computed)
        output = model(image)
        # 3. Calculating the loss value
        loss = loss_func(output, labels)
        # 4. Calculating new grdients given the loss value
        loss.backward()
        # 5. Updating the weights
        optimizer.step()
        # 6. logging (Optional)
        if batch_idx % vis_step == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(image),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.data.item()))
    # Validation Phase on train dataset
    for batch_idx, (image, labels) in enumerate(train_loader):
        output = model(image)
        _ , pred_label = output.data.max(dim=1)
        num_hit += (pred_label == labels.data).sum()
    train_accuracy = (num_hit.item() / total)
    print("Epoch: {}, Training Accuracy: {:.2f}%".format(epoch, 100. * train_accuracy))

    if scheduler:
        scheduler.step(train_accuracy)
    
    return 100. * train_accuracy

In [104]:
def eval_model_val(model, val_loader, epoch):
    num_hit = 0
    total = len(test_loader.dataset)

    for batch_idx, (image, labels) in enumerate(val_loader): # Complete the rest of this function
        output = model(image)
        _ , pred_label = output.data.max(dim=1)
        num_hit += (pred_label == labels.data).sum()
    test_accuracy = (num_hit.item() / total)
    print("Epoch: {}, Validation Accuracy: {:.2f}%".format(epoch, 100. * test_accuracy))
    return 100. * test_accuracy 


In [105]:
def eval_model_test(model, test_loader, epoch):
    num_hit = 0
    total = len(test_loader.dataset)
    all_preds = []
    all_labels = []

    for batch_idx, (image, labels) in enumerate(test_loader):
        output = model(image)
        _, pred_label = output.data.max(dim=1)
        num_hit += (pred_label == labels.data).sum()
        all_preds.extend(pred_label.cpu().numpy())
        all_labels.extend(labels.data.cpu().numpy())

    test_accuracy = (num_hit.item() / total)
    print("Epoch: {}, Testing Accuracy: {:.2f}%".format(epoch, 100. * test_accuracy))
    return 100. * test_accuracy, all_preds, all_labels

def final_evaluation(all_preds, all_labels):
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print("Final Evaluation Metrics:")
    print("Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(precision*100, recall*100, f1*100))

    # Generate confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(conf_matrix)


    # Analyze confusion matrix for class-specific performance
    class_accuracy = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
    for i, acc in enumerate(class_accuracy):
        print("Class {} Accuracy: {:.2f}%".format(i, 100. * acc))

In [106]:
def runModel(model, optimizer, scheduler):
    torch.manual_seed(0)
    # Training Hyperparameters
    batch_size = 32 
    learning_rate = 0.01
    itr = 20 
    
    # for running on gpu
    if cuda:
        model = model.cuda()
    
    # 2. Initialize model's weight
    model.apply(weights_init)
    
    # 3. Define optimizer and loss function
    loss_func = torch.nn.CrossEntropyLoss()
    
    # 4. Write the training loop
    train_acc = []
    test_acc = []
    total_time = 0
    all_preds = []
    all_labels = []
    for epoch in range(itr):
        start = time()
        tr_acc = train_model(model, optimizer, train_loader, loss_func, epoch+1, scheduler)
        vs_acc = eval_model_val(model, val_loader, epoch+1)
        ts_acc, epoch_preds, epoch_labels = eval_model_test(model, test_loader,epoch+1)
        train_acc.append(tr_acc)
        test_acc.append(ts_acc)
        all_preds.extend(epoch_preds)
        all_labels.extend(epoch_labels)
        end = time()
        total_time += end-start
    print("Training and evaluation finished in:", total_time, "sec.")
    final_evaluation(all_preds, all_labels)


transform type =1
we resize

In [20]:
model = ConvNet(image_size , 1)
optimizer = optim.SGD(params = model.parameters(), lr = learning_rate)
runModel(model, optimizer, None )

Train Epoch: 1 [0/601 (0%)]	Loss: 2.703585
Train Epoch: 1 [80/601 (13%)]	Loss: 1.284705
Train Epoch: 1 [160/601 (26%)]	Loss: 1.306450
Train Epoch: 1 [240/601 (39%)]	Loss: 1.356872
Train Epoch: 1 [320/601 (53%)]	Loss: 1.572598
Train Epoch: 1 [400/601 (66%)]	Loss: 0.938165
Train Epoch: 1 [480/601 (79%)]	Loss: 1.244697
Train Epoch: 1 [560/601 (92%)]	Loss: 0.948339
Epoch: 1, Training Accuracy: 51.25%
Epoch: 1, Validation Accuracy: 50.00%
Epoch: 1, Testing Accuracy: 53.85%
Train Epoch: 2 [0/601 (0%)]	Loss: 1.160579
Train Epoch: 2 [80/601 (13%)]	Loss: 1.115921
Train Epoch: 2 [160/601 (26%)]	Loss: 1.079053
Train Epoch: 2 [240/601 (39%)]	Loss: 1.278095
Train Epoch: 2 [320/601 (53%)]	Loss: 1.947712
Train Epoch: 2 [400/601 (66%)]	Loss: 0.961217
Train Epoch: 2 [480/601 (79%)]	Loss: 1.104916
Train Epoch: 2 [560/601 (92%)]	Loss: 0.985849
Epoch: 2, Training Accuracy: 59.07%
Epoch: 2, Validation Accuracy: 58.97%
Epoch: 2, Testing Accuracy: 57.69%
Train Epoch: 3 [0/601 (0%)]	Loss: 1.059262
Train Epoch

In [33]:
model = ConvNet(image_size , 2)
learning_rate = 0.01
optimizer = optim.SGD(params = model.parameters(), lr = learning_rate)
runModel(model, optimizer, None )

Train Epoch: 1 [0/601 (0%)]	Loss: 2.472919
Train Epoch: 1 [80/601 (13%)]	Loss: 1.501659
Train Epoch: 1 [160/601 (26%)]	Loss: 1.279527
Train Epoch: 1 [240/601 (39%)]	Loss: 1.310018
Train Epoch: 1 [320/601 (53%)]	Loss: 1.153296
Train Epoch: 1 [400/601 (66%)]	Loss: 1.187111
Train Epoch: 1 [480/601 (79%)]	Loss: 1.420956
Train Epoch: 1 [560/601 (92%)]	Loss: 1.096948
Epoch: 1, Training Accuracy: 47.59%
Epoch: 1, Validation Accuracy: 42.31%
Epoch: 1, Testing Accuracy: 48.72%
Train Epoch: 2 [0/601 (0%)]	Loss: 1.028374
Train Epoch: 2 [80/601 (13%)]	Loss: 1.035772
Train Epoch: 2 [160/601 (26%)]	Loss: 1.007012
Train Epoch: 2 [240/601 (39%)]	Loss: 1.350958
Train Epoch: 2 [320/601 (53%)]	Loss: 0.866009
Train Epoch: 2 [400/601 (66%)]	Loss: 1.218280
Train Epoch: 2 [480/601 (79%)]	Loss: 1.269423
Train Epoch: 2 [560/601 (92%)]	Loss: 1.230453
Epoch: 2, Training Accuracy: 49.42%
Epoch: 2, Validation Accuracy: 41.03%
Epoch: 2, Testing Accuracy: 48.72%
Train Epoch: 3 [0/601 (0%)]	Loss: 0.798735
Train Epoch

In [35]:
learning_rate = 0.01
model = ConvNet(image_size , 3)
optimizer = optim.SGD(params = model.parameters(), lr = learning_rate)
runModel(model, optimizer, None )

Train Epoch: 1 [0/601 (0%)]	Loss: 3.292532
Train Epoch: 1 [80/601 (13%)]	Loss: 1.376858
Train Epoch: 1 [160/601 (26%)]	Loss: 1.404992
Train Epoch: 1 [240/601 (39%)]	Loss: 1.386560
Train Epoch: 1 [320/601 (53%)]	Loss: 1.425201
Train Epoch: 1 [400/601 (66%)]	Loss: 1.407784
Train Epoch: 1 [480/601 (79%)]	Loss: 1.355771
Train Epoch: 1 [560/601 (92%)]	Loss: 1.364277
Epoch: 1, Training Accuracy: 29.28%
Epoch: 1, Validation Accuracy: 28.21%
Epoch: 1, Testing Accuracy: 32.05%
Train Epoch: 2 [0/601 (0%)]	Loss: 1.394343
Train Epoch: 2 [80/601 (13%)]	Loss: 1.376668
Train Epoch: 2 [160/601 (26%)]	Loss: 1.369573
Train Epoch: 2 [240/601 (39%)]	Loss: 1.483323
Train Epoch: 2 [320/601 (53%)]	Loss: 1.385440
Train Epoch: 2 [400/601 (66%)]	Loss: 1.382094
Train Epoch: 2 [480/601 (79%)]	Loss: 1.317208
Train Epoch: 2 [560/601 (92%)]	Loss: 1.335960
Epoch: 2, Training Accuracy: 29.28%
Epoch: 2, Validation Accuracy: 28.21%
Epoch: 2, Testing Accuracy: 32.05%
Train Epoch: 3 [0/601 (0%)]	Loss: 1.410843
Train Epoch

C:\Users\u5588649\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
learning_rate = 0.01
model = ConvNet(image_size , 4)
optimizer = optim.SGD(params = model.parameters(), lr = learning_rate)
runModel(model, optimizer, None )

Train Epoch: 1 [0/601 (0%)]	Loss: 4.758243
Train Epoch: 1 [80/601 (13%)]	Loss: 1.125328
Train Epoch: 1 [160/601 (26%)]	Loss: 1.191504
Train Epoch: 1 [240/601 (39%)]	Loss: 1.542684
Train Epoch: 1 [320/601 (53%)]	Loss: 1.445540
Train Epoch: 1 [400/601 (66%)]	Loss: 1.198609
Train Epoch: 1 [480/601 (79%)]	Loss: 1.113742
Train Epoch: 1 [560/601 (92%)]	Loss: 1.120309
Epoch: 1, Training Accuracy: 39.10%
Epoch: 1, Validation Accuracy: 43.59%
Epoch: 1, Testing Accuracy: 44.87%
Train Epoch: 2 [0/601 (0%)]	Loss: 1.260209
Train Epoch: 2 [80/601 (13%)]	Loss: 1.106963
Train Epoch: 2 [160/601 (26%)]	Loss: 0.896620
Train Epoch: 2 [240/601 (39%)]	Loss: 1.403111
Train Epoch: 2 [320/601 (53%)]	Loss: 1.049507
Train Epoch: 2 [400/601 (66%)]	Loss: 1.077821
Train Epoch: 2 [480/601 (79%)]	Loss: 1.281857
Train Epoch: 2 [560/601 (92%)]	Loss: 0.829710
Epoch: 2, Training Accuracy: 42.60%
Epoch: 2, Validation Accuracy: 43.59%
Epoch: 2, Testing Accuracy: 44.87%
Train Epoch: 3 [0/601 (0%)]	Loss: 1.166939
Train Epoch

In [107]:
model = ConvNet(image_size , 1)
lr_ADAMb = 0.001
optimizer = optim.Adam(params = model.parameters(), lr = lr_ADAMb)
runModel(model, optimizer, None )

Train Epoch: 1 [0/601 (0%)]	Loss: 2.703585
Train Epoch: 1 [80/601 (13%)]	Loss: 1.214542
Train Epoch: 1 [160/601 (26%)]	Loss: 1.323028
Train Epoch: 1 [240/601 (39%)]	Loss: 1.286260
Train Epoch: 1 [320/601 (53%)]	Loss: 1.332494
Train Epoch: 1 [400/601 (66%)]	Loss: 0.945579
Train Epoch: 1 [480/601 (79%)]	Loss: 1.191540
Train Epoch: 1 [560/601 (92%)]	Loss: 0.891938
Epoch: 1, Training Accuracy: 55.91%
Epoch: 1, Validation Accuracy: 56.41%
Epoch: 1, Testing Accuracy: 55.13%
Train Epoch: 2 [0/601 (0%)]	Loss: 1.043403
Train Epoch: 2 [80/601 (13%)]	Loss: 1.068133
Train Epoch: 2 [160/601 (26%)]	Loss: 0.812672
Train Epoch: 2 [240/601 (39%)]	Loss: 1.106075
Train Epoch: 2 [320/601 (53%)]	Loss: 1.326671
Train Epoch: 2 [400/601 (66%)]	Loss: 1.003737
Train Epoch: 2 [480/601 (79%)]	Loss: 0.573176
Train Epoch: 2 [560/601 (92%)]	Loss: 0.598474
Epoch: 2, Training Accuracy: 64.89%
Epoch: 2, Validation Accuracy: 66.67%
Epoch: 2, Testing Accuracy: 71.79%
Train Epoch: 3 [0/601 (0%)]	Loss: 0.814861
Train Epoch

In [111]:
model = ConvNet(image_size , 1)
lr_ADAMb = 0.001
optimizer = optim.Adam(params = model.parameters(), lr = lr_ADAMb)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)
runModel(model, optimizer, scheduler )

Train Epoch: 1 [0/601 (0%)]	Loss: 2.703585
Train Epoch: 1 [80/601 (13%)]	Loss: 1.214542
Train Epoch: 1 [160/601 (26%)]	Loss: 1.323028
Train Epoch: 1 [240/601 (39%)]	Loss: 1.286260
Train Epoch: 1 [320/601 (53%)]	Loss: 1.332494
Train Epoch: 1 [400/601 (66%)]	Loss: 0.945579
Train Epoch: 1 [480/601 (79%)]	Loss: 1.191540
Train Epoch: 1 [560/601 (92%)]	Loss: 0.891938
Epoch: 1, Training Accuracy: 55.91%
Epoch: 1, Validation Accuracy: 56.41%
Epoch: 1, Testing Accuracy: 55.13%
Train Epoch: 2 [0/601 (0%)]	Loss: 1.043403
Train Epoch: 2 [80/601 (13%)]	Loss: 1.068133
Train Epoch: 2 [160/601 (26%)]	Loss: 0.812672
Train Epoch: 2 [240/601 (39%)]	Loss: 1.106075
Train Epoch: 2 [320/601 (53%)]	Loss: 1.326671
Train Epoch: 2 [400/601 (66%)]	Loss: 1.003737
Train Epoch: 2 [480/601 (79%)]	Loss: 0.573176
Train Epoch: 2 [560/601 (92%)]	Loss: 0.598474
Epoch: 2, Training Accuracy: 64.89%
Epoch: 2, Validation Accuracy: 66.67%
Epoch: 2, Testing Accuracy: 71.79%
Train Epoch: 3 [0/601 (0%)]	Loss: 0.814861
Train Epoch

### 2.2	Improving performance through data augmentation (15 marks)

In [123]:
cuda = False
torch.manual_seed(0)
if torch.cuda.is_available() and cuda:
    torch.cuda.manual_seed_all(0)
    FloatType = torch.cuda.FloatTensor
    LongType = torch.cuda.LongTensor
else:
    FloatType = torch.FloatTensor
    LongType = torch.LongTensor

In [124]:
# Model Hyperparameters
image_size = 32*32
num_classes = 62
num_hidden_unit = 100

# Training Hyperparameters (Note: These values are not the optimal ones)
batch_size = 32 
learning_rate = 0.1
itr = 20 

In [125]:
# Define transformation
transforms = None
augment = True

transforms_regular = trans.Compose([trans.Grayscale(), trans.Resize([32,32]), trans.ToTensor(), trans.Normalize(mean=(0.5,), std = (0.5,))])

data_augmentation = trans.Compose([trans.Grayscale(),
    trans.RandomRotation(50),  # Rotate first to prevent cropping issues
    trans.RandomResizedCrop(32, scale=(0.6, 1.2)),  # Then randomly crop & resize
    trans.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.7, 1.3)),  # Shift & scale
    trans.RandomPerspective(distortion_scale=0.2, p=0.5),  # Perspective distortions
    trans.RandomHorizontalFlip(0.6),  # Flip signs if applicable
    trans.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),  # Blur to simulate low-quality images
    trans.ColorJitter(brightness=0.3, contrast=0.3),  # Adjust brightness/contrast
    trans.ToTensor(), trans.Normalize(mean=(0.5,), std = (0.5,))
])


if augment is True :
    transforms = data_augmentation
else:
    transforms = transforms_regular

In [126]:
# Loading dataset
class BelgiumTSCDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.paths = glob(os.path.join(self.root, '**', "*.png"))

    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        img = self.transform(Image.open(path))
        label = int(path.split(os.path.sep)[-2])
        return img, label
    
train_data = BelgiumTSCDataset(root='./data/BelgiumTSC_Training/Training', transform= transforms)
test_data = BelgiumTSCDataset(root='./data/BelgiumTSC_Testing/Testing', transform= transforms)

subtest_data = BelgiumTSCDataset(root='./data/Test_subset', transform= transforms_regular)

# Create DataLoader
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True, num_workers = 0)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = False, num_workers = 0)
subtest_loader = torch.utils.data.DataLoader(subtest_data, batch_size = batch_size, shuffle = False, num_workers = 0)

In [127]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight.data)
        m.bias.data.normal_(mean=0,std=1e-2)
    elif isinstance(m, nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight.data)
        m.bias.data.normal_(mean=0,std=1e-2)

In [128]:
def train_model2(model, optimizer, train_loader, loss_func, epoch, vis_step = 20):
    # Number of samples with correct classification
    num_hit = 0
    # total size of train data
    total = len(train_loader.dataset)
    # Training loop over batches of data on train dataset
    for batch_idx, (image, labels) in enumerate(train_loader):
        # 1. Clearing previous gradient values.
        optimizer.zero_grad()
        # 2. feeding images to model (forward method will be computed)
        output = model(image)
        # 3. Calculating the loss value
        loss = loss_func(output, labels)
        # 4. Calculating new grdients given the loss value
        loss.backward()
        # 5. Updating the weights
        optimizer.step()
        # 6. logging (Optional)
        if batch_idx % vis_step == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(image),
                                                                           len(train_loader.dataset),
                                                                           100. * batch_idx / len(train_loader),
                                                                           loss.data.item()))
    # Validation Phase on train dataset
    for batch_idx, (image, labels) in enumerate(train_loader):
        output = model(image)
        _ , pred_label = output.data.max(dim=1)
        num_hit += (pred_label == labels.data).sum()
    train_accuracy = (num_hit.item() / total)
    print("Epoch: {}, Training Accuracy: {:.2f}%".format(epoch, 100. * train_accuracy))
    return 100. * train_accuracy

In [129]:
def eval_model(model, test_loader, epoch, name):
    num_hit = 0
    total = len(test_loader.dataset)
    for batch_idx, (image, labels) in enumerate(test_loader): # Complete the rest of this function
        output = model(image)
        _ , pred_label = output.data.max(dim=1)
        num_hit += (pred_label == labels.data).sum()
    test_accuracy = (num_hit.item() / total)
    print("Epoch: {}, {}Testing Accuracy: {:.2f}%".format(epoch,name, 100. * test_accuracy))
    return 100. * test_accuracy 


In [130]:
class ConvNetNew(nn.Module):
    
    def __init__(self):
        super(ConvNetNew, self).__init__() 
        self.conv1 = nn.Conv2d(in_channels=1,
        	                   out_channels=6,
            				   kernel_size=(3,3),
            				   stride=(1,1))
        self.pool2 = nn.MaxPool2d(kernel_size = (2,2))
        self.conv3 = nn.Conv2d(in_channels=6,
                               out_channels=31,
                               kernel_size=(2,2),
                               stride=(1,1))# Complete this line
        self.pool4 = nn.MaxPool2d(kernel_size = (2,2)) # Complete this line
        self.conv5 = nn.Conv2d(in_channels=31,
                               out_channels=93,
                               kernel_size=(2,2),
                               stride=(1,1)) # Complete this line
        self.pool6 = nn.MaxPool2d(kernel_size = (2,2)) # Complete this line
        self.conv7 = nn.Conv2d(in_channels=93,
                               out_channels=186,
                               kernel_size=(3,3),
                               stride=(1,1)) # Complete this line
        self.fc8 = nn.Linear(in_features=186,
            			     out_features=124)
        self.fc9 = nn.Linear(in_features=124,
            			     out_features=93)
        self.fc10 = nn.Linear(in_features=93,
                             out_features=62) # Complete this line

        
    def forward(self,x):
        x = x.view([-1, 1, 32, 32])
        x = nn.functional.relu(self.conv1(x))
#         print(x.shape)
        x = self.pool2(x)
#         print(x.shape)
        x = nn.functional.relu(self.conv3(x))
#         print(x.shape)
        x = self.pool4(x)
#         print(x.shape)
        x = nn.functional.relu(self.conv5(x))
#         print(x.shape)
        x = self.pool6(x)
#         print(x.shape)
        x = nn.functional.relu(self.conv7(x))
#         print(x.shape)
        x = x.view(-1, 186)
#         print(x.shape)
        x = nn.functional.relu(self.fc8(x))
#         print(x.shape)
        x = nn.functional.relu(self.fc9(x))
#         print(x.shape)
        x = self.fc10(x)
        
        return x

In [131]:
def dataAugTest():
    torch.manual_seed(0)
    # 1. Instantiate from the model class 
    model = ConvNetNew(
                     )
    
    # for running on gpu
    if cuda:
        model = model.cuda()
    
    # 2. Initialize model's weight
    model.apply(weights_init)
    
    # 3. Define optimizer and loss function
    lr_ADAMb = 0.001
    optimizer = optim.Adam(params = model.parameters(), lr = lr_ADAMb)
    loss_func = torch.nn.CrossEntropyLoss()
    
     # 4. Write the training loop
    train_acc = []
    test_acc = []
    subtest_acc = []
    total_time = 0
    for epoch in range(itr):
        start = time()
        tr_acc = train_model2(model, optimizer, train_loader, loss_func, epoch+1)
        ts_acc = eval_model(model, test_loader, epoch+1, '')
        subts_acc = eval_model(model, subtest_loader, epoch+1, 'Sub-')
        train_acc.append(tr_acc)
        test_acc.append(ts_acc)
        subtest_acc.append(subts_acc)
        end = time()
        total_time += end-start
    print("Training and evaluation finished in:", total_time, "sec.")


In [54]:
dataAugTest()

4575
Train Epoch: 1 [0/4575 (0%)]	Loss: 5.583474
Train Epoch: 1 [640/4575 (14%)]	Loss: 3.971108
Train Epoch: 1 [1280/4575 (28%)]	Loss: 3.235387
Train Epoch: 1 [1920/4575 (42%)]	Loss: 3.506693
Train Epoch: 1 [2560/4575 (56%)]	Loss: 2.212386
Train Epoch: 1 [3200/4575 (70%)]	Loss: 2.070684
Train Epoch: 1 [3840/4575 (84%)]	Loss: 1.980191
Train Epoch: 1 [4480/4575 (98%)]	Loss: 1.938052
Epoch: 1, Training Accuracy: 59.17%
Epoch: 1, Testing Accuracy: 56.55%
Epoch: 1, Sub-Testing Accuracy: 3.19%
4575
Train Epoch: 2 [0/4575 (0%)]	Loss: 1.474632
Train Epoch: 2 [640/4575 (14%)]	Loss: 1.920446
Train Epoch: 2 [1280/4575 (28%)]	Loss: 1.059073
Train Epoch: 2 [1920/4575 (42%)]	Loss: 1.443858
Train Epoch: 2 [2560/4575 (56%)]	Loss: 1.273787
Train Epoch: 2 [3200/4575 (70%)]	Loss: 1.068045
Train Epoch: 2 [3840/4575 (84%)]	Loss: 0.769453
Train Epoch: 2 [4480/4575 (98%)]	Loss: 1.025923
Epoch: 2, Training Accuracy: 74.69%
Epoch: 2, Testing Accuracy: 71.55%
Epoch: 2, Sub-Testing Accuracy: 1.20%
4575
Train Epo

In [132]:
dataAugTest()

Train Epoch: 1 [0/4575 (0%)]	Loss: 6.198489
Train Epoch: 1 [640/4575 (14%)]	Loss: 3.948425
Train Epoch: 1 [1280/4575 (28%)]	Loss: 3.636053
Train Epoch: 1 [1920/4575 (42%)]	Loss: 3.857497
Train Epoch: 1 [2560/4575 (56%)]	Loss: 3.284226
Train Epoch: 1 [3200/4575 (70%)]	Loss: 3.633670
Train Epoch: 1 [3840/4575 (84%)]	Loss: 3.504374
Train Epoch: 1 [4480/4575 (98%)]	Loss: 3.536265
Epoch: 1, Training Accuracy: 9.86%
Epoch: 1, Testing Accuracy: 7.34%
Epoch: 1, Sub-Testing Accuracy: 4.78%
Train Epoch: 2 [0/4575 (0%)]	Loss: 3.336920
Train Epoch: 2 [640/4575 (14%)]	Loss: 3.338401
Train Epoch: 2 [1280/4575 (28%)]	Loss: 3.614360
Train Epoch: 2 [1920/4575 (42%)]	Loss: 3.432230
Train Epoch: 2 [2560/4575 (56%)]	Loss: 3.368360
Train Epoch: 2 [3200/4575 (70%)]	Loss: 3.498532
Train Epoch: 2 [3840/4575 (84%)]	Loss: 3.213641
Train Epoch: 2 [4480/4575 (98%)]	Loss: 3.289824
Epoch: 2, Training Accuracy: 19.32%
Epoch: 2, Testing Accuracy: 21.23%
Epoch: 2, Sub-Testing Accuracy: 3.19%
Train Epoch: 3 [0/4575 (0%